1. Давайте уточним правило трёх сигм. Утверждение: 99.7% вероятностной массы случайной величины X∼N(μ,σ2) лежит в интервале μ±c⋅σ. Чему равно точное значение константы c? Округлите ответ до четырёх знаков после десятичной точки.

In [1]:
from scipy.stats import norm

In [2]:
alpha = 0.003
norm.ppf(1 - alpha/2)

2.9677379253417944

2. В пятилетнем рандомизированном исследовании Гарвардской медицинской школы 11037 испытуемых через день принимали аспирин, а ещё 11034 — плацебо. Исследование было слепым, то есть, испытуемые не знали, что именно они принимают. За 5 лет инфаркт случился у 104 испытуемых, принимавших аспирин, и у 189 принимавших плацебо. Оцените, насколько вероятность инфаркта снижается при приёме аспирина. Округлите ответ до четырёх знаков после десятичной точки.

In [3]:
aspirin = 11037
placebo = 11034
asp_inf = 104
plac_inf = 189

In [4]:
p_a = asp_inf / aspirin
p_p = plac_inf / placebo

In [5]:
p_p - p_a

0.0077060239760047815

3. Постройте теперь 95% доверительный интервал для снижения вероятности инфаркта при приёме аспирина. Чему равна его верхняя граница? Округлите ответ до четырёх знаков после десятичной точки.

In [6]:
from statsmodels.stats.proportion import proportion_confint
import numpy as np

In [7]:
interval = proportion_confint(asp_inf, aspirin)
interval

(0.007620422638288568, 0.011225278186210842)

In [8]:
def proportions_confint_diff_ind(p1, p2, len1, len2, alpha = 0.05):    
    z = norm.ppf(1 - alpha / 2.)   
    
    left_boundary = (p1 - p2) - z * np.sqrt(p1 * (1 - p1)/ len1 + p2 * (1 - p2)/ len2)
    right_boundary = (p1 - p2) + z * np.sqrt(p1 * (1 - p1)/ len1 + p2 * (1 - p2)/ len2)
    
    return (left_boundary, right_boundary)

In [9]:
proportions_confint_diff_ind(p_p, p_a, placebo, aspirin)

(0.004687750675049439, 0.010724297276960124)

4. Продолжим анализировать данные эксперимента Гарвардской медицинской школы.

In [10]:
odds_placebo = p_p / (1 - p_p)
odds_placebo

0.017427385892116183

In [11]:
odds_aspirin = p_a / (1 - p_a)
odds_aspirin

0.009512485136741973

In [12]:
odds_placebo / odds_aspirin

1.8320539419087138

5. Величина, которую вы оценили в предыдущем вопросе, называется отношением шансов. Постройте для отношения шансов 95% доверительный интервал с помощью бутстрепа. Чему равна его нижняя граница? Округлите ответ до 4 знаков после десятичной точки.

Чтобы получить в точности такой же доверительный интервал, как у нас:

- составьте векторы исходов в контрольной и тестовой выборках так, чтобы в начале шли все единицы, а потом все нули;
- установите random seed=0;
- сделайте по 1000 псевдовыборок из каждой группы пациентов с помощью функции get_bootstrap_samples.

In [13]:
aspirin_data = np.zeros(aspirin)
aspirin_data[:asp_inf] = 1
placebo_data = np.zeros(placebo)
placebo_data[:plac_inf] = 1

In [14]:
def get_bootstrap_samples(data, n_samples):
    indices = np.random.randint(0, len(data), (n_samples, len(data)))
    samples = data[indices]
    return samples

In [15]:
def stat_intervals(stat, alpha):
    boundaries = np.percentile(stat, [100 * alpha / 2., 100 * (1 - alpha / 2.)])
    return boundaries

In [16]:
np.random.seed(0)

#получим списки вероятностей
aspirin_mean_scores = list(map(np.mean, get_bootstrap_samples(aspirin_data, 1000)))
placebo_mean_scores = list(map(np.mean, get_bootstrap_samples(placebo_data, 1000)))

print( "95% confidence interval for the aspirin median repair time:",  stat_intervals(aspirin_mean_scores, 0.05))
print( "95% confidence interval for the placebo median repair time:",  stat_intervals(placebo_mean_scores, 0.05))

95% confidence interval for the aspirin median repair time: [0.00752016 0.01123494]
95% confidence interval for the placebo median repair time: [0.01477252 0.01957586]


In [17]:
#получим списки шансов
aspirin_odds_list = [p / (1 - p) for p in aspirin_mean_scores]
placebo_odds_list = [p / (1 - p) for p in placebo_mean_scores]

In [18]:
#посчитаем отношения шансов
ratio_mean_scores = list(map(lambda x: x[1] / x[0], zip(aspirin_odds_list, placebo_odds_list)))
print( "95% confidence interval for the difference between odds",  stat_intervals(ratio_mean_scores, 0.05))

95% confidence interval for the difference between odds [1.44419465 2.34321168]
